In [1]:
import gym
import random

In [2]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n #number of actios we can take

In [4]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [8]:
episodes = 10
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score: {}'.format(episode, score))
env.close()

Episode:0 Score: 120.0
Episode:1 Score: 80.0
Episode:2 Score: 45.0
Episode:3 Score: 110.0
Episode:4 Score: 110.0
Episode:5 Score: 170.0
Episode:6 Score: 110.0
Episode:7 Score: 80.0
Episode:8 Score: 90.0
Episode:9 Score: 210.0


In [10]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [13]:
def create_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [21]:
del model

In [23]:
model = create_model(height, width, channels, actions)

In [24]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               34603520  
_________________________________________________________________
dense_11 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_12 (Dense)             (None, 6)                

In [25]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [28]:
def create_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg',
                  nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [29]:
dqn = create_agent(model, actions)
dqn.compile(Adam(lr=1e-4))
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...
  426/10000: episode: 1, duration: 12.024s, episode steps: 426, steps per second:  35, episode reward: 130.000, mean reward:  0.305 [ 0.000, 30.000], mean action: 2.559 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 1234/10000: episode: 2, duration: 328.658s, episode steps: 808, steps per second:   2, episode reward: 225.000, mean reward:  0.278 [ 0.000, 30.000], mean action: 2.447 [0.000, 5.000],  loss: 10.058667, mean_q: 3.340725, mean_eps: 0.899470
 1886/10000: episode: 3, duration: 875.061s, episode steps: 652, steps per second:   1, episode reward: 105.000, mean reward:  0.161 [ 0.000, 25.000], mean action: 2.604 [0.000, 5.000],  loss: 1.109316, mean_q: 3.140183, mean_eps: 0.859645
 2705/10000: episode: 4, duration: 1060.202s, episode steps: 819, steps per second:   1, episode reward: 155.000, mean reward:  0.189 [ 0.000, 30.000], mean action: 2.503 [0.000, 5.000],  loss: 0.637999, mean_q: 3.071300, mean_eps: 0.793450
 3385/10000: episode: 5, dur

In [30]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...


c:\users\filip\appdata\local\programs\python\python38\lib\site-packages\gym\envs\atari\environment.py:255: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(


Episode 1: reward: 145.000, steps: 705
Episode 2: reward: 260.000, steps: 985
Episode 3: reward: 140.000, steps: 782
Episode 4: reward: 110.000, steps: 689
Episode 5: reward: 285.000, steps: 1423
Episode 6: reward: 185.000, steps: 845
Episode 7: reward: 110.000, steps: 681
Episode 8: reward: 25.000, steps: 375
Episode 9: reward: 210.000, steps: 817
Episode 10: reward: 150.000, steps: 663
162.0


In [32]:
dqn.save_weights('savedModels/10k-weights.h5f')

[WARNING] savedModels/10k-weights.h5f.index already exists - overwrite? [y/n]y
[TIP] Next time specify overwrite=True!


In [ ]:
dqn.load_weights('savedModels/10k-weights.h5f')